## Automated anisotropic resistivity inversion for efficient formation evaluation and uncertainty quantification

### Misael M. Morales, Ali Eghbali, Oriyomi Raheem, Michael Pyrcz, Carlos Torres-Verdin
***
## KAN-based Inversion (Kolmogorov-Arnold Networks)
***

In [ ]:
from main import *

check_torch()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
case1, case2, synthetic1, synthetic2 = load_all_data()

In [ ]:
from kan import KAN, create_dataset

In [ ]:
f = lambda x: torch.exp(torch.sin(torch.pi*x[:,[0]]) + x[:,[1]]**2)
dataset = create_dataset(f, n_var=2)
print(dataset['train_input'].shape, dataset['train_label'].shape)
print(dataset.keys())

In [ ]:
model = KAN(width=[2,5,1], grid=5, k=3)
model.train(dataset, opt='LBFGS', steps=20, lamb=0.1)
model.plot()

In [ ]:
y_true = dataset['train_label'].detach().numpy()
y_pred = model(dataset['train_input']).detach().numpy()
print('True: {} | Pred: {}'.format(y_true.shape, y_pred.shape))

In [ ]:
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
ax.scatter(dataset['train_label'], y_pred)
ax.axline([0,0], [1,1], color='k', ls='--')
ax.grid(True, which='both', alpha=0.4)
ax.set(xlabel='True', ylabel='Pred', title='Prediction', xlim=(0,7.5), ylim=(0,7.5))
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15,5))
ax1, ax2, ax3 = axs

im1 = ax1.scatter(dataset['train_input'][:,0], dataset['train_input'][:,1], edgecolor='lightgrey',
                    c=dataset['train_label'].squeeze(), vmin=0, vmax=7.5)
ax1.set(xlabel='x', ylabel='y', title='f(x,y) | true')
ax1.grid(True, which='both', alpha=0.4)
cb1 = fig.colorbar(im1, ax=ax1)

im2 = ax2.scatter(dataset['train_input'][:,0], dataset['train_input'][:,1], edgecolor='lightgrey',
                    c=y_pred, vmin=0, vmax=7.5)
ax2.set(xlabel='x', ylabel='y', title='f(x,y) | predicted')
ax2.grid(True, which='both', alpha=0.4)
cb2 = fig.colorbar(im2, ax=ax2)

im3 = ax3.scatter(dataset['train_input'][:,0], dataset['train_input'][:,1], edgecolor='lightgrey',
                c=dataset['train_label'].squeeze()-y_pred.squeeze(), cmap='seismic', vmin=-1, vmax=1)
ax3.set(xlabel='x', ylabel='y', title='f(x,y) | difference')
ax3.grid(True, which='both', alpha=0.4)
cb3 = fig.colorbar(im3, ax=ax3)

plt.tight_layout()
plt.show()

***
***

In [ ]:
idx = np.random.choice(len(case1), int(len(case1)*0.8), replace=False)
zdx = case1.index[idx]

jdx = np.setdiff1d(np.arange(len(case1)), idx)
wdx = case1.index[jdx]

In [ ]:
mydataset = {'train_input':torch.tensor(case1.loc[zdx,['Rv','Rh']].values), 
             'train_label':torch.tensor(case1.loc[zdx,['Rv','Rh']].values),
             'test_input':torch.tensor(case1.loc[wdx,['Rv','Rh']].values),
             'test_label':torch.tensor(case1.loc[wdx,['Rv','Rh']].values)}
print('Train:', mydataset['train_input'].shape, mydataset['train_label'].shape)
print('Test:', mydataset['test_input'].shape, mydataset['test_label'].shape)

In [ ]:
model = KAN(width=[2,5,2], grid=5, k=3)
model.train(mydataset, steps=1);
model.plot()

y_train_pred = model(mydataset['train_input']).detach().numpy()
y_test_pred = model(mydataset['test_input']).detach().numpy()

In [ ]:
y_pred = np.concatenate([y_train_pred, y_test_pred], axis=0)
print(y_pred.shape)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 7), sharey=True)
ax1, ax2 = axs


ax1.plot(case1['Rv'], case1.index, c='darkred', label='True Rv')
ax1.plot(y_pred[:,0], case1.index, c='k', ls='--', alpha=0.85, label='Predicted Rv')

ax2.plot(case1['Rh'], case1.index, c='darkblue', label='True Rh')
ax2.plot(y_pred[:,1], case1.index, c='k', ls='--', alpha=0.85, label='Predicted Rh')

[ax.set(xscale='log') for ax in axs]
[ax.grid(True, which='both', alpha=0.4) for ax in axs]
[ax.legend() for ax in axs]
ax1.invert_yaxis()
plt.tight_layout()
plt.show()

***
# END